In [1]:
import sys

sys.path.append("../quicfire_tools")
from quicfire_tools import outputs
from quicfire_tools.parameters import SimulationParameters

In [2]:
import zarr
import numpy as np
import xarray as xr
import dask.array as da
from rechunker import rechunk
from pathlib import Path
from IPython.core.display import display, HTML

/var/folders/_z/54qnvh3d5ll86lchz81zm3dr0000gn/T/ipykernel_6266/1572318097.py:7: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [3]:
DATA_PATH = Path("../tests/data")
SIMULATION_PATH = DATA_PATH / "crazy-canyon-simulation"
OUTPUT_PATH = SIMULATION_PATH / "Output"
DRAWFIRE_PATH = OUTPUT_PATH / "drawfire"

# Create simulation parameters object
SIM_PARAMS = SimulationParameters(
    nx=100,
    ny=100,
    nz=56,
    dx=1,
    dy=1,
    dz=1,
    wind_speed=6.5,
    wind_direction=270,
    sim_time=600,
    auto_kill=0,
    num_cpus=4,
    fuel_flag=4,
    ignition_flag=1,
    output_time=100,
    topo_flag=0,
)

In [4]:
sim_outputs = outputs.SimulationOutputs(OUTPUT_PATH, SIM_PARAMS)

In [5]:
zarr_simulation_outputs = sim_outputs.to_zarr(DATA_PATH / "tmp/test.zarr")

In [6]:
print(zarr_simulation_outputs.tree())

/
 ├── fire-energy_to_atmos (2, 100, 100, 57) float64
 ├── fire-reaction_rate (2, 100, 100, 57) float64
 ├── fuels-dens (2, 100, 100, 57) float64
 ├── fuels-moist (2, 100, 100, 57) float64
 ├── groundfuelheight (1, 100, 100, 1) float64
 ├── mburnt_integ (2, 100, 100, 1) float64
 ├── thermaldose (2, 100, 100, 57) float64
 ├── thermalradiation (2, 100, 100, 57) float64
 └── windu (2, 100, 100, 57) float64


# Xarray rechunking

### All arrays in a dataset

In [7]:
ds_3d = xr.open_zarr(DATA_PATH / "tmp/test.zarr", drop_variables=["groundfuelheight", "mburnt_integ"], consolidated=False)

In [8]:
print(ds_3d)

<xarray.Dataset>
Dimensions:               (time: 2, ny: 100, nx: 100, nz: 57)
Dimensions without coordinates: time, ny, nx, nz
Data variables:
    fire-energy_to_atmos  (time, ny, nx, nz) float64 dask.array<chunksize=(1, 100, 100, 57), meta=np.ndarray>
    fire-reaction_rate    (time, ny, nx, nz) float64 dask.array<chunksize=(1, 100, 100, 57), meta=np.ndarray>
    fuels-dens            (time, ny, nx, nz) float64 dask.array<chunksize=(1, 100, 100, 57), meta=np.ndarray>
    fuels-moist           (time, ny, nx, nz) float64 dask.array<chunksize=(1, 100, 100, 57), meta=np.ndarray>
    thermaldose           (time, ny, nx, nz) float64 dask.array<chunksize=(1, 100, 100, 57), meta=np.ndarray>
    thermalradiation      (time, ny, nx, nz) float64 dask.array<chunksize=(1, 100, 100, 57), meta=np.ndarray>
    windu                 (time, ny, nx, nz) float64 dask.array<chunksize=(1, 100, 100, 57), meta=np.ndarray>


In [47]:
ds_3d["fuels-dens"]

<xarray.DataArray 'fuels-dens' (time: 2, ny: 100, nx: 100, nz: 57)>
dask.array<open_dataset-fuels-dens, shape=(2, 100, 100, 57), dtype=float64, chunksize=(1, 100, 100, 57), chunktype=numpy.ndarray>
Dimensions without coordinates: time, ny, nx, nz

In [10]:
# Rechunk the full dataset along the time dimension
ds_3d_rechunked = ds_3d.chunk({"time": 2})

In [11]:
print(ds_3d_rechunked)

<xarray.Dataset>
Dimensions:               (time: 2, ny: 100, nx: 100, nz: 57)
Dimensions without coordinates: time, ny, nx, nz
Data variables:
    fire-energy_to_atmos  (time, ny, nx, nz) float64 dask.array<chunksize=(2, 100, 100, 57), meta=np.ndarray>
    fire-reaction_rate    (time, ny, nx, nz) float64 dask.array<chunksize=(2, 100, 100, 57), meta=np.ndarray>
    fuels-dens            (time, ny, nx, nz) float64 dask.array<chunksize=(2, 100, 100, 57), meta=np.ndarray>
    fuels-moist           (time, ny, nx, nz) float64 dask.array<chunksize=(2, 100, 100, 57), meta=np.ndarray>
    thermaldose           (time, ny, nx, nz) float64 dask.array<chunksize=(2, 100, 100, 57), meta=np.ndarray>
    thermalradiation      (time, ny, nx, nz) float64 dask.array<chunksize=(2, 100, 100, 57), meta=np.ndarray>
    windu                 (time, ny, nx, nz) float64 dask.array<chunksize=(2, 100, 100, 57), meta=np.ndarray>


In [48]:
ds_3d_rechunked["fuels-dens"]

<xarray.DataArray 'fuels-dens' (time: 2, ny: 100, nx: 100, nz: 57)>
dask.array<rechunk-merge, shape=(2, 100, 100, 57), dtype=float64, chunksize=(2, 100, 100, 57), chunktype=numpy.ndarray>
Dimensions without coordinates: time, ny, nx, nz

### Rechunk a single data array in the dataset 

In [51]:
da_fuels_dens = ds_3d["fuels-dens"]
da_fuels_dens

<xarray.DataArray 'fuels-dens' (time: 2, ny: 100, nx: 100, nz: 57)>
dask.array<open_dataset-fuels-dens, shape=(2, 100, 100, 57), dtype=float64, chunksize=(1, 100, 100, 57), chunktype=numpy.ndarray>
Dimensions without coordinates: time, ny, nx, nz

In [50]:
da_fuels_dens_rechunked = da_fuels_dens.chunk({"nx": 10, "ny": 10, "nz": 10})
da_fuels_dens_rechunked

<xarray.DataArray 'fuels-dens' (time: 2, ny: 100, nx: 100, nz: 57)>
dask.array<rechunk-merge, shape=(2, 100, 100, 57), dtype=float64, chunksize=(1, 10, 10, 10), chunktype=numpy.ndarray>
Dimensions without coordinates: time, ny, nx, nz

### Just the mburnt_integ array in a dataset

In [53]:
output_name = "mburnt_integ"
dropped_variables = [var for var in zarr_simulation_outputs if var != output_name]
ds_2d = xr.open_zarr(
    DATA_PATH / "tmp/test.zarr",
    drop_variables=dropped_variables,
    consolidated=False,
)

In [54]:
print(ds_2d)

<xarray.Dataset>
Dimensions:       (time: 2, y: 100, x: 100, z: 1)
Dimensions without coordinates: time, y, x, z
Data variables:
    mburnt_integ  (time, y, x, z) float64 dask.array<chunksize=(1, 100, 100, 1), meta=np.ndarray>


In [55]:
ds_2d["mburnt_integ"]

<xarray.DataArray 'mburnt_integ' (time: 2, y: 100, x: 100, z: 1)>
dask.array<open_dataset-mburnt_integ, shape=(2, 100, 100, 1), dtype=float64, chunksize=(1, 100, 100, 1), chunktype=numpy.ndarray>
Dimensions without coordinates: time, y, x, z

In [56]:
# Rechunk the ds along the time dimension
ds_2d_rechunked = ds_2d.chunk({"time": -1})

In [57]:
print(ds_2d_rechunked)

<xarray.Dataset>
Dimensions:       (time: 2, y: 100, x: 100, z: 1)
Dimensions without coordinates: time, y, x, z
Data variables:
    mburnt_integ  (time, y, x, z) float64 dask.array<chunksize=(2, 100, 100, 1), meta=np.ndarray>


In [58]:
ds_2d_rechunked["mburnt_integ"]

<xarray.DataArray 'mburnt_integ' (time: 2, y: 100, x: 100, z: 1)>
dask.array<rechunk-merge, shape=(2, 100, 100, 1), dtype=float64, chunksize=(2, 100, 100, 1), chunktype=numpy.ndarray>
Dimensions without coordinates: time, y, x, z

# Dask rechunking

In [59]:
dask_array = sim_outputs.to_dask("mburnt_integ")

dask.array<setitem, shape=(2, 100, 100, 1), dtype=float64, chunksize=(1, 100, 100, 1), chunktype=numpy.ndarray>

In [37]:
display(HTML(dask_array._repr_html_()))

In [38]:
dask_array_rechunked = dask_array.rechunk((2, 100, 100, 57))

In [40]:
display(HTML(dask_array_rechunked._repr_html_()))